### import

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm, trange
from math import exp, sqrt, log
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.special import i0, i1, iv
from numpy import random
from torch.nn.functional import normalize
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from torchsummary import summary

### Define the model of the neural network

In [5]:
class BASICnet(nn.Module):
    def __init__(self, dim, width):
        super(BASICnet, self).__init__()
        self.dim = dim
        self.width = width

        self.fc1 = nn.Linear(self.dim, self.width)
        self.fc2 = nn.Linear(self.width, self.width)
        self.fc3 = nn.Linear(self.width, self.width)
        self.fc4 = nn.Linear(self.width, self.width)

        self.output = nn.Linear(self.width, 1, bias=True)

    def forward(self, x):
        # s = nn.ConstantPad2d(x, (0, self.dim - self.dim))
        y = self.fc1(x)
        y = torch.tanh(y)
        y = self.fc2(y)
        y = torch.tanh(y)
        y = self.fc3(y)
        y = torch.tanh(y)
        y = self.fc4(y)
        y = torch.tanh(y)
        
        output = self.output(y)
        return output

    def assign_value(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                nn.init.constant_(m.bias.data, 0.1)

class TESTNet(nn.Module):
    def __init__(self):
        super(TESTNet, self).__init__()
        self.fc1 = nn.Linear(10, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 1)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

### Cauculate equation (Righthand side of the equation, RHS)

### Generate model from the dataset and using montecarlo method to train the model

In [212]:
import random

def calculate_fx(X):
    # Check if there are boundary cases with 1 and -1 in each row
    has_boundary = torch.any((X == 1) | (X == -1), dim=-1)

    # Create an initial fx tensor filled with the regular values
    regular_fx = -160 * (torch.tensor(4 * np.pi, dtype=torch.float32).pow(2)) * torch.prod(torch.sin(4 * np.pi * X), dim=-1)
    
    # Where there are boundary cases, set fx to zero
    fx = torch.where(has_boundary, torch.tensor(0.0, dtype=torch.float32, device=X.device), regular_fx)

    return fx



def data_generator_monte_carlo(Numberofgenpoints, dim, prob_special = 0.1):
    source = torch.randn(size = (Numberofgenpoints, dim), requires_grad=True) # 生成一个大矩阵
    source = normalize(source, p=2)
    radius = torch.rand(size = (Numberofgenpoints, 1))
    radius = torch.pow(torch.rand(size = (Numberofgenpoints, 1)), 1/dim)
    source = source * radius

    # # 随即确认替换的行
    num_replace_incol = torch.randint(1, Numberofgenpoints, size=(1,)).item()

    for _ in range(num_replace_incol):
        if random.random() < prob_special:
            special_value = random.choice([1, -1])
            row_index = torch.randint(0, Numberofgenpoints, size=(1,)).item()
            source[row_index] = torch.full((dim, ), special_value)
        else:
            row_index = torch.randint(0, Numberofgenpoints, size=(1,)).item()
            col_index = torch.randint(0, dim, size=(1,)).item()
            replace_value = torch.randint(0, 2, size=(1,)).item() * 2 - 1
            source[row_index, col_index] = replace_value

    max_value = torch.max(source)
    min_value = torch.min(source)

    target = calculate_fx(source)
    # set target as the requres_graade true

    if max_value > 1 or min_value < -1:
        raise ValueError
    
    return source, target

### Dataset generate

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [213]:
from torch.utils.data import TensorDataset, DataLoader
test_gen = 10000
test_dim = 10

test_source, test_target = data_generator_monte_carlo(test_gen, test_dim)
test_source = torch.tensor(test_source).to(device)
test_target = torch.tensor(test_target).to(device)
train_tensor = TensorDataset(test_source, test_target)


/tmp/ipykernel_588706/4016173177.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_source = torch.tensor(test_source).to(device)
/tmp/ipykernel_588706/4016173177.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_target = torch.tensor(test_target).to(device)


In [45]:
import numpy as np

# 定义边界点的数量
num_boundary_points = 10000  # 根据需要选择数量

# 生成均匀分布的边界点
boundary_points = np.random.uniform(-1, 1, size=(num_boundary_points, 10))

# 将边界点限制在边界上
boundary_points = np.where(boundary_points < -0.3, -1, boundary_points)
boundary_points = np.where(boundary_points > 0.3, 1, boundary_points)

# 创建一个包含边界点的数据集
boundary_points = torch.tensor(boundary_points, dtype=torch.float32)


## The optimizer and scheduler

In [214]:
model = BASICnet(10, 100)
model.to(device)

BASICnet(
  (fc1): Linear(in_features=10, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=100, bias=True)
  (output): Linear(in_features=100, out_features=1, bias=True)
)

flag1

In [215]:
min_loss = 100000
dim = 10
LR = 1e-3
BATCHSIZE = 10000
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = StepLR(optimizer, step_size=1000, gamma=0.8)

training_size = int(0.8 * len(train_tensor))
validation_size = len(train_tensor) - training_size
train_dataset, validation_dataset = torch.utils.data.random_split(train_tensor, [training_size, validation_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCHSIZE, shuffle=True)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=BATCHSIZE, shuffle=False)

boundary_loader = DataLoader(dataset=boundary_points, batch_size=BATCHSIZE, shuffle=True)
total_iterations = 10000
loss = nn.MSELoss(reduction='sum')
loss_all, loss_1, loss_2, loss_3, loss_4 = [], [], [], [], []
val_losses = []
for epoch in trange(total_iterations):
    mse, mse_1, mse_2, mse_3 = 0, 0, 0, 0
    val_loss = 0
    for X, psi_true in train_loader:
        optimizer.zero_grad()
        
        X.requires_grad = True
        psi_pred = model(X).squeeze()
        l1 = loss(psi_pred, psi_true)
        
        d_psi = torch.autograd.grad(psi_pred, X, torch.ones_like(psi_pred), create_graph=True)[0]
        d_psi_sum = torch.sum(d_psi, dim=1);
        d_psi_2 = torch.autograd.grad(d_psi_sum, X, torch.ones_like(d_psi_sum), create_graph=True)[0]
        d_psi_2_sum = torch.sum(d_psi_2, dim=1);
        l2 = loss(d_psi_2_sum, psi_true)
        
        for X_bo in boundary_loader:
            X_bo.requires_grad = True
            X_bo = X_bo.to(device)
            psi_pred_bo = model(X_bo).squeeze()
            d_psi_bo = torch.autograd.grad(psi_pred_bo, X_bo, torch.ones_like(psi_pred_bo), create_graph=True)[0]
            d_psi_bo_sum = torch.sum(d_psi_bo, dim=1);
            d_psi_bo2 = torch.autograd.grad(d_psi_bo_sum, X_bo, torch.ones_like(d_psi_bo_sum), create_graph=True)[0]
            d_psi_bo2_sum = torch.sum(d_psi_bo2, dim=1);
            bc = torch.zeros_like(d_psi_bo2_sum)
            l3 = loss(d_psi_bo2_sum, bc)

        l1 /= X.shape[0]
        l2 /= X.shape[0]
        l3 /= X_bo.shape[0]
        
        l = l2 + l3
        
        l.backward()
        mse += l.item()
        mse_1 += l1.item()
        mse_2 += l2.item()
        mse_3 += l3.item()

        optimizer.step()

    mse /= BATCHSIZE
    mse_1 /= BATCHSIZE
    mse_2 /= BATCHSIZE
    mse_3 /= BATCHSIZE


    scheduler.step()
    loss_all.append(mse)
    loss_1.append(mse_1)
    loss_2.append(mse_2)
    loss_3.append(mse_3)
    
    if min_loss > mse*1.2:
        improvement = min_loss - mse
        min_loss = mse
        torch.save(model.state_dict(), 'model.pth')
        print('Improve: ' + str(improvement) + ' Save MSE Model: ' + str(mse))

    # for X_val, psi_true_val in validation_loader:
    #         X_val = X_val.to(device)
    #         X_val.requires_grad = True
    #         psi_true_val = psi_true_val.to(device)
    #         psi_pred_val = model(X_val).squeeze()
    
    #         # 计算二阶微分
    #         d_psi_val = torch.autograd.grad(psi_pred_val, X_val, torch.ones_like(psi_pred_val), create_graph=True)[0]
    #         d_psi_2_val = torch.autograd.grad(d_psi_val, X_val, torch.ones_like(d_psi_val), create_graph=True)[0]
    #         d_psi_2_sum_val = torch.sum(d_psi_2_val, dim=1)
    #         # 计算自定义损失
    #         val_loss += loss(d_psi_2_sum_val, psi_true_val)

    # val_loss /= len(validation_loader.dataset)  # 计算验证损失的平均值
    # val_losses.append(val_loss)
    # print('Validation Loss: ' + str(val_loss))


    # if epoch%10 == 0:
        print('Epoch: ' + str(epoch) + ' MSE: ' + str(mse) + 
              ' MSE 1: ' + str(mse_1) + ' MSE 2: ' + str(mse_2) + 
              ' MSE 3: ' + str(mse_3))
        



  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 1/10000 [00:00<22:45,  7.32it/s]

Improve: 99977.4774234375 Save MSE Model: 22.5225765625
Epoch: 0 MSE: 22.5225765625 MSE 1: 22.5225296875 MSE 2: 22.5225765625 MSE 3: 3.062847245018929e-08


  1%|          | 107/10000 [00:19<25:35,  6.44it/s]

Improve: 3.7898703124999997 Save MSE Model: 18.73270625
Epoch: 105 MSE: 18.73270625 MSE 1: 22.5634625 MSE 2: 18.621153125 MSE 3: 0.111553857421875


...

Improve: 0.03731833496093748 Save MSE Model: 0.1862869873046875
Epoch: 3512 MSE: 0.1862869873046875 MSE 1: 22.59533125 MSE 2: 0.1210887939453125 MSE 3: 0.06519819946289063


 42%|████▏     | 4174/10000 [12:06<14:23,  6.74it/s]

Improve: 0.03108156738281251 Save MSE Model: 0.155205419921875
Epoch: 4173 MSE: 0.155205419921875 MSE 1: 22.5941421875 MSE 2: 0.09815601806640625 MSE 3: 0.057049407958984374


 46%|████▌     | 4607/10000 [13:20<15:37,  5.75it/s]


KeyboardInterrupt: 

In [221]:
target = X_val[2:3]
target2 = X_bo[2:3]

RHL = calculate_fx(target)
VAL = model(target)

BO = model(target2)
RHL2 = calculate_fx(target2)


grad = torch.autograd.grad(VAL, target, create_graph=True)
grad_sum = torch.sum(grad[0], dim=1)
grad_2 = torch.autograd.grad(grad_sum, target, create_graph=True)
grad_2_sum = torch.sum(grad_2[0], dim=1)

grad_bo = torch.autograd.grad(BO, target2, create_graph=True)
grad_sum_bo = torch.sum(grad_bo[0], dim=1)
grad_2_bo = torch.autograd.grad(grad_sum_bo, target2, create_graph=True)
grad_2_sum_bo = torch.sum(grad_2_bo[0], dim=1)
print(VAL)
print(X_val[0:1])
print(X_val[0:1].shape)
print('Right Hand Eqution: ',RHL)
print('Left Hand Eqution: ',grad_2_sum)

print('BO RHL : ', RHL2)
print('BO LHL: ',grad_2_sum_bo)
# print(VAL)
# print(VAL.shape)

# print(grad_sum)
# print(grad_2_sum)


tensor([[-4.1844]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[ 0.0489, -0.4484, -0.4159, -0.0293, -0.2300, -0.5179, -0.0168,  0.1121,
          0.2104, -0.1248]], device='cuda:0', grad_fn=<SliceBackward0>)
torch.Size([1, 10])
Right Hand Eqution:  tensor([563.5513], device='cuda:0', grad_fn=<SWhereBackward0>)
Left Hand Eqution:  tensor([499.2750], device='cuda:0', grad_fn=<SumBackward1>)
BO RHL :  tensor([0.], device='cuda:0', grad_fn=<SWhereBackward0>)
BO LHL:  tensor([-31.9384], device='cuda:0', grad_fn=<SumBackward1>)


flag

In [200]:
# 保存整个模型
# torch.save(model, 'model.pth')
torch.load('model.pth')